# torchattacks demo_Imagenet data

In [2]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torch.nn.functional as F
from torchvision.utils import save_image
import torchattacks
import os


## Preprocessing

### Load pretraiend model vgg19 from torchvision

In [3]:
# Here is the selection: Loading from Github or local source.

#model = torch.hub.load("pytorch/vision:v0.13.0", "vgg16_bn", pretrained=True)
model=torchvision.models.vgg19(pretrained=True)

c:\Users\19401\anaconda3\envs\PRP\lib\site-packages\torchvision\models\_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
c:\Users\19401\anaconda3\envs\PRP\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Load dataset from imagenet

In [4]:
# based on 3060
use_cuda = True

norm_mean = [0.485, 0.456, 0.406]
norm_std = [0.229, 0.224, 0.225]
transform = transforms.Compose(
    [transforms.Resize((256, 256)),
     transforms.ToTensor(),
     transforms.Normalize(norm_mean, norm_std),
])



testset = torchvision.datasets.ImageFolder(root='../../../imagenet2012/',
                                       transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64,
                                         shuffle=False)


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

org_img = []
org_labels = []
with torch.no_grad():
    for data in testloader:
        images, labels = data
        org_img.append(images)
        org_labels.append(labels)



### Test accuracy of the model

In [18]:
# definition of const
# Small test
st = 100
# Midium test
mt = 10
# Large test
lt = 20
# Full test
ft = 100

In [16]:
def test_acc_new(model, testloader, test_images, n_image, save_test_images = False):
    # n_image is the scale of test_dataset
    # As the result of terrible consumption of memory by Imagenet, here we don't store each label, but just calculate the acc
    correct = 0
    total = 0
    p_labels = []

    # The final target is images -> Remeber to save the img
    # TODO: image_save

    saved_img = []
    i = 0
    for data in testloader:
        images, labels = data
        if total < n_image:
            outputs = model(test_images[i])
            _, predicted = torch.max(outputs.data, 1)
            for j in labels:
                if predicted[total%64] == j:
                    correct += 1
                total += 1
            i += 1
        else:
            break
    test_accuracy = (100.0 * correct / total)
    print('Accuracy of the network on the', total, "images is: ", test_accuracy, '%')
    
    if save_test_images == True:
        return test_accuracy, p_labels, saved_img
    else:
        return test_accuracy, p_labels,total

In [17]:
test_accuracy, vgg_labels,num = test_acc_new(model, testloader, org_img, st, False)

Accuracy of the network on the 1024 images is:  88.28125 %


In [ ]:
print(org_labels)


In [10]:
print(vgg_labels)

[]


## Attack

### Tool functions

In [11]:
def unnormalize(img, mean = np.array(norm_mean), std = np.array(norm_std)):
    '''
    unnormalize the image that has been normalized with mean and std
    '''
    inverse_mean = - mean/std
    inverse_std = 1/std
    img = transforms.Normalize(mean=-mean/std, std=1/std)(img)
    return img

def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))

def ax_imshow(ax, img, label):
    img = unnormalize(img)     # unnormalize
    img = np.clip(img, 0., 1.)
    ax.set_xticks([])
    ax.set_yticks([])
    img = np.transpose(img, (1,2,0))
    ax.imshow(img)
    ax.set_title(classes[label])

### Implement torchattacks.FGSM

In [12]:
test_loader_1=torch.utils.data.DataLoader(testset,batch_size=1,shuffle=True)
epsilons = []
init_eps = 0
accuracies=[]
step = 0.001
n_step = 5


In [13]:
# Just a single test
epsilons=[0.001]

In [ ]:
eps_times=0
fin_image=[]
fgsm_examples = []
for eps in epsilons:
    visual_examples = 5
    fgsm_img = []
    attack = torchattacks.FGSM(model, eps)
    count = 0
    for i in range(5):
        print(i)
        fgsm_img.append(attack(org_img[i], org_labels[i]))
    fgsm_test_accuracy, fgsm_labels , a_images= test_acc_new(model, testloader, fgsm_img, st, True)
    dataiter = iter(testloader)
    images, labels = dataiter.next()
    accuracies.append(fgsm_test_accuracy)

### Conclusion

In [ ]:
plt.figure(figsize=(5,5))
plt.plot([0] + epsilons, [test_accuracy] + accuracies, "*-")
plt.yticks(np.arange(0.0, test_accuracy, step=10))
plt.xticks(np.arange(0.0, max(epsilons), step=max(epsilons)/5))
plt.title("Accuracy vs Epsilon")
plt.xlabel("Epsilon")
plt.ylabel("Accuracy")
plt.show()

Save image

In [ ]:
# for i in range(len(fgsm_examples)):
#     dir_name=os.getcwd()+'/FGSM_img/epi_%i/'%(i,)
#     os.mkdir(dir_name)
#     id=0
#     for j in range(len(fgsm_examples[i])):
#         save_image(fgsm_examples[i][j],dir_name+'_%i.png'%(id,))
#         id+=1

In [ ]:
dataiter = iter(testloader)
images, labels = dataiter.next()

# print images
imshow(torchvision.utils.make_grid(a_images[0]))
print('GroundTruth: ', ' '.join('%5s' % classes[labels[j]] for j in range(4)))

In [ ]:
imshow(torchvision.utils.make_grid(fgsm_examples[6][1]))

In [ ]:
imshow(torchvision.utils.make_grid(orig[1]))